# Working With Large Datasets:PLACES LOCAL Data for Better Health.

The purpose of this project is to accomplish two main tasks. 
1) Test Whether or not States with more PLACES locations have different means that those with fewer locations, and how they differ exactly.
2) Sucessfully work with very large data sets: Cleaning, organizinng and performing analyisis on them. 


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statistics
import scipy.stats as stats
from distfit import distfit
import statsmodels.api as sm 
import pylab
from numpy import cov
from itertools import permutations 
import itertools
from scipy.stats import spearmanr
from random import sample 
import random 
from collections import Counter
from tabulate import tabulate
import requests
import zipfile
import io
import os  
from scipy.stats import iqr
import scikit_posthocs as sp
pd.set_option('display.max_colwidth', None)

# Data Background, Loading, Inspection


There are many ways to acess the files that are needed. Either downloading from github, \
or if the files are too big  we can use the shortcuts themselves. And download right from the source. \
The files can be found below. The next few cells load in the data and put it into the current working directory. 



https://catalog.data.gov/dataset/places-local-data-for-better-health-place-data-2022-release/resource/4bfea8ab-534b-4f2b-9cb1-d0b951709c2a 

https://catalog.data.gov/dataset/places-local-data-for-better-health-place-data-2021-release-06a9b/resource/f9bb4b0d-7db2-432e-bf3e-58bd400a6ffc 

https://catalog.data.gov/dataset/places-local-data-for-better-health-place-data-2020-release-670b7/resource/b95272f6-c03d-487b-81fa-460d4b9bcb1f 





In [ ]:
current_directory = os.getcwd()

print('Downloading started')
url = 'https://github.com/oohtmeel1/Data_Mining_Final/raw/main/s2020.zip'

# Downloading the file by sending the request to the URL
req = requests.get(url)

# Check if the request was successful (status code 200)
if req.status_code == 200:
    # Create a BytesIO object from the content
    zip_content = io.BytesIO(req.content)

    # Get the current working directory
    current_directory = os.getcwd()
    print(f'Current Working Directory: {current_directory}')

    # Specify the directory where you want to extract the contents
    extraction_path = 'zipped_files_now'  
    zip_ref = zipfile.ZipFile(zip_content, 'r')

    # Create the extraction directory if it doesn't exist
    os.makedirs(extraction_path, exist_ok=True)

    # Join the extraction path and the filename
    extraction_file_path = os.path.join(extraction_path, 's2020.zip')

    # Extract the contents
    zip_ref.extractall(extraction_file_path)
    zip_ref.close()

In [ ]:
extraction_path = 'zipped_files_now'
# Specify the filename 
csv_filename = 's2020.csv'  # Replace with the actual CSV file name

# Join the extraction path and the filename
csv_file_path = os.path.join(extraction_path, csv_filename)


    # Read the CSV file into a DataFrame
s2020 = pd.read_csv(csv_file_path)

In [ ]:


url = 'https://data.cdc.gov/api/views/q8ig-wwk9/rows.csv?accessType=DOWNLOAD'

# Downloading the file by sending the request to the URL
req = requests.get(url)

# Check if the request was successful 
if req.status_code == 200:
    # Create a BytesIO object from the content
    s2021 = pd.read_csv(io.StringIO(req.text))


In [ ]:
import requests
import pandas as pd

url = ' https://data.cdc.gov/api/views/epbn-9bv3/rows.csv?accessType=DOWNLOAD'

# Downloading the file by sending the request to the URL
req = requests.get(url)

# Check if the request was successful 
if req.status_code == 200:
    # Create a BytesIO object from the content
    s2022 = pd.read_csv(io.StringIO(req.text))

In [ ]:
s2020.head(1)


In [ ]:
s2021.head(1)

In [ ]:
s2022.head(1)

# Some Basic Data Cleaning
All NA values are dropped. And all rows and columns have been inspected to ensure correctness.The only null values remaining are data value footnotes(Which are NaN) but are of no importance. Otherwise the Data Value Column does not containy any null values. And looking at the info shows there are no very large values.Which is a good sign that there are no random digits present.

In [ ]:
# Drop NA values
s2020['Measure'] = s2020['Measure'].str.replace(" ", "_") # Strip whitespace and replace with _
s2021['Measure'] = s2021['Measure'].str.replace(" ", "_") # Strip whitespace and replace with _
s2022['Measure'] = s2022['Measure'].str.replace(" ", "_") # Strip whitespace and replace with _
s2020= s2020[s2020['Data_Value'].notna()]
s2021= s2021[s2021['Data_Value'].notna()]
s2022= s2022[s2022['Data_Value'].notna()]

In [ ]:
np.where(pd.isnull(s2020['Data_Value'])) # Returns where the dataframe is null

In [ ]:
np.where(pd.isnull(s2021['Data_Value']))

In [ ]:
np.where(pd.isnull(s2022['Data_Value']))

In [ ]:
s2020.loc[s2020['Data_Value'].idxmax()]

In [ ]:
s2021.loc[s2021['Data_Value'].idxmax()]

In [ ]:
s2022.loc[s2022['Data_Value'].idxmax()]

# Some basic visualizations
How are the locations distributed for each state?
The number of unique locations are counted and plotted.

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].nunique()
city_counts1 = city_counts1.sort_values(ascending=False)


fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts1,palette="viridis")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")


In [ ]:
city_counts2 = s2021.groupby('StateAbbr')['LocationName'].nunique()
city_counts2 = city_counts2.sort_values(ascending=False)



fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts2,palette="viridis")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2021")

In [ ]:
city_counts3 = s2022.groupby('StateAbbr')['LocationName'].nunique()
city_counts3 = city_counts3.sort_values(ascending=False)



fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts3,palette="viridis")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2022")

In [ ]:

measure_counts = s2020.groupby(['StateAbbr', 'Measure']).size().reset_index(name='Count')

The below plot answers the obvious. Do places with more locations measured overall have a higher number of Data points obtained. Yes, the answer is yes. They also look fairly consistent year to year.

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='StateAbbr', y='Count', data=measure_counts, dodge=True,legend=False)
plt.xlabel('State Abbreviation')
plt.ylabel('Count')
plt.title('Count of Measures in Each State 2020')

In [ ]:

measure_counts1 = s2021.groupby(['StateAbbr', 'Measure']).size().reset_index(name='Count')

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='StateAbbr', y='Count', data=measure_counts1, dodge=True,legend=False)
plt.xlabel('State Abbreviation')
plt.ylabel('Count')
plt.title('Count of Measures in Each State 2021')

In [ ]:

measure_counts3 = s2022.groupby(['StateAbbr', 'Measure']).size().reset_index(name='Count')

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='StateAbbr', y='Count', data=measure_counts3, dodge=True,legend=False)
plt.xlabel('State Abbreviation')
plt.ylabel('Count')
plt.title('Count of Measures in Each State 2022')

# How are values divided up as far as category goes?
How are the values divided up as far as Category  goes. There are 3. PREVENT, HLTHOUT, abd UNHBEH.

HLTHOUT always have the most, PREVENT seems to be second and UNHBEH usually has the smallest number of entries.

In [ ]:

measure_counts5 = s2020.groupby(['StateAbbr', 'CategoryID']).size().reset_index(name='Count')

In [ ]:
measure_counts6 = s2021.groupby(['StateAbbr', 'CategoryID']).size().reset_index(name='Count')

In [ ]:
measure_counts7 = s2022.groupby(['StateAbbr', 'CategoryID']).size().reset_index(name='Count')

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='StateAbbr', y='Count', data=measure_counts5, hue ='CategoryID',dodge=True,legend=True)
plt.xlabel('State Abbreviation')
plt.ylabel('Count')
plt.title('Count of Measures in Each State')

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='StateAbbr', y='Count', data=measure_counts6, hue ='CategoryID',dodge=True,legend=True)
plt.xlabel('State Abbreviation')
plt.ylabel('Count')
plt.title('Count of Measures in Each State')

In [ ]:
plt.figure(figsize=(20, 6))
sns.barplot(x='StateAbbr', y='Count', data=measure_counts7, hue ='CategoryID',dodge=True,legend=True)
plt.xlabel('State Abbreviation')
plt.ylabel('Count')
plt.title('Count of Measures in Each State')

# Code to split large dataframes into smaller chunks
It is really cumbersome trying to work with these dataframes as they are. So I am going to split them up into smaller bits.\
Working with such a large amounts of data can be a little bit unwieldy at first. 
So while it is not recommended for normal use, in this case \
I used the exec and eval functions to split the dataframes into smaller chunks\
Storing those in a list of dataframes. 


In [ ]:

#Getting measures by themselves and creating just a ton of df
sa=s2020['StateAbbr'].unique()  # Getting all names of all sates
n = len(sa)
new_list_of_titles = []
dfs = []
for i in sa:
	if i not in new_list_of_titles:
		new_list_of_titles.append(i)
print(new_list_of_titles)


grouped = s2020.groupby(s2020['StateAbbr']) # Group by year
for i in new_list_of_titles:
	exec(f'{i}=grouped.get_group("{i}")')  # Using f string here to execute code. Getting the groups of the grouped well.groups. and executing based on state. So I make 51 dataframes this way one for each
dataframes_dict = {}

# Create DataFrames and store them in the dictionary 
for i in new_list_of_titles:
	dataframes_dict[i] = eval(i)
# put all of them into a list so I can maybe iterateover them easier Use this for the iteration
#eval(f'dataframes_list_{year} = list(dataframes_dict.values())') # no one sees me using exec. Nope. 
dataframes_list2020 = list(dataframes_dict.values())
	

In [ ]:
	
sa=s2021['StateAbbr'].unique()  # Getting all names of all sates
n = len(sa)
new_list_of_titles = []
dfs = []
for i in sa:
	if i not in new_list_of_titles:
		new_list_of_titles.append(i)
print(new_list_of_titles)


grouped = s2021.groupby(s2021['StateAbbr']) # Group by year
for i in new_list_of_titles:
	exec(f'{i}=grouped.get_group("{i}")')  # Using f string here to execute code. Getting the groups of the grouped well.groups. and executing based on state. So I make 51 dataframes this way one for each
dataframes_dict = {}

# Create DataFrames and store them in the dictionary 
for i in new_list_of_titles:
	dataframes_dict[i] = eval(i)
# put all of them into a list so I can maybe iterateover them easier Use this for the iteration
#eval(f'dataframes_list_{year} = list(dataframes_dict.values())') # no one sees me using exec. Nope. 
dataframes_list2021 = list(dataframes_dict.values())

In [ ]:
# Getting measures by themselves and creating just a ton of df
sa=s2022['StateAbbr'].unique()  # Getting all names of all sates
n = len(sa)
new_list_of_titles = []
dfs = []
for i in sa:
	if i not in new_list_of_titles:
		new_list_of_titles.append(i)
print(new_list_of_titles)


grouped = s2022.groupby(s2022['StateAbbr']) # Group by year
for i in new_list_of_titles:
	exec(f'{i}=grouped.get_group("{i}")')  # Using f string here to execute code. Getting the groups of the grouped well.groups. and executing based on state. So I make 51 dataframes this way one for each
dataframes_dict = {}

# Create DataFrames and store them in the dictionary 
for i in new_list_of_titles:
	dataframes_dict[i] = eval(i)
# put all of them into a list so I can maybe iterateover them easier Use this for the iteration
#eval(f'dataframes_list_{year} = list(dataframes_dict.values())') # no one sees me using exec. Nope. 
dataframes_list2022 = list(dataframes_dict.values())


The new lists of dataframes can use indexing to access where each State is. For example:

In [ ]:
dataframes_list2020[0].head(1)

In [ ]:
dataframes_list2021[0].head(1)

In [ ]:
dataframes_list2022[0].head(1)

# How to deal with outliers with large amounts of data
There are many ways to approach this problem. The biggest factor is time\
larger datasets require more time and care since it is easier to miss values that may not belong or outliers.\
While histograms and QQ plots are still useful, it might be better to use some python built-ins to help \
and to save time. 
The percentile functions will be used to perform this. A summary is given below.

The code below runs the summary statistics for all unique measures in the entire dataframe by State.
And in order to remove outliers we can use the ``` IQR ``` method. \
Where we set up a 'fence' outside of Q1 and Q3. Anything outside of this fence is considered an outlier.\
The formula for calculating the 'fence' is :
$$\begin{align}
((1.5 * IQR) - Q1)  
\end{align}$$
OR 
$$\begin{align}
((1.5 * IQR) + Q3) 
\end{align}$$


In [ ]:
abc=dataframes_list2020[50]
len(abc.loc[abc['Measure']=='Chronic_obstructive_pulmonary_disease_among_adults_aged_>=18_years','Data_Value'])

In [ ]:
x_sum=[]
y=[]
n = len(dataframes_list2020)
#n=2
new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2020 = dataframes_list2020[j] # matches 2020 is first dataframe
	subseta=(matches_2020['StateAbbr'].unique()) # Just a list of the state names in that df
	#print(subseta)
	matching_indices1 = [index for index, df in enumerate(dataframes_list2020) if (df['StateAbbr'].isin(subseta).any())]# No matter where they are, this could should find the matching states. 
	for index in matching_indices1:
		matches_2020 = dataframes_list2020[index]
		subset1=(matches_2020['Measure'].unique()) 
		intersection_list = set(list(subset1))
		for i in intersection_list:
			lis1 = matches_2020.loc[matches_2020['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
			#print(len(lis1))
			my_array = np.array(lis1)
			q1,q3 = np.percentile(my_array, (25,75))
			iqr1 = q3-q1		
			#print(iqr)
			min1 = q1-(1.5*iqr1) # Subtract from Q1
			#(print(min1))
			max1 = (1.5*iqr1)+q3 # Add to Q3
			x_sum.append([subseta,min1,max1,i,q1,q3])
			#print("Total prints:", print_count)

In [ ]:
List_of_min_max=pd.DataFrame(x_sum) # Put those in a dataframe
List_of_min_max['StateAbbr'] = List_of_min_max[0].astype(str)
List_of_min_max['StateAbbr'] = List_of_min_max['StateAbbr'].str.replace("['", '').str.replace("']", '')
List_of_min_max["StateAbbr"] = List_of_min_max["StateAbbr"].str.strip()
List_of_min_max['min1'] = List_of_min_max[1].astype(float)
List_of_min_max['max1'] = List_of_min_max[2].astype(float)
List_of_min_max['Measure'] = List_of_min_max[3].astype(str)
List_of_min_max['Measure'] = List_of_min_max['Measure'].str.strip()
List_of_min_max['Q1'] = List_of_min_max[4].astype(float)
List_of_min_max['Q3'] = List_of_min_max[5].astype(float)
List_of_min_max= List_of_min_max.drop([0,1,2,3,4,5],axis=1)

In [ ]:
okee=s2020[s2020['StateAbbr']=='NC']
abc2=pd.DataFrame(okee.loc[okee['Measure']=='Older_adult_women_aged_>=65_years_who_are_up_to_date_on_a_core_set_of_clinical_preventive_services:_Flu_shot_past_year,_PPV_shot_ever,_Colorectal_cancer_screening,_and_Mammogram_past_2_years','Data_Value'])
abc2.describe()

In [ ]:
experimentdf=dataframes_list2020[0] # Manually Checking My data
experimentdf1=dataframes_list2020[1] 
experimentdf2=dataframes_list2020[2] 

In [ ]:
experimentdf.head(1)

In [ ]:
experimentdf1.head(1)

In [ ]:
column_names=s2020.columns.values.tolist()
dataframes_list2020_1 =[] #<- New Dataframe list
n = len(dataframes_list2020)

result_df = pd.DataFrame()
for j in range(n):

	matches_2020 = dataframes_list2020[j] # <- Change this matches 2020 is first dataframe
	matches_2020['Measure'] = matches_2020['Measure'].str.strip()
	matches_2020['StateAbbr']=matches_2020['StateAbbr'].str.strip()
	matches_2020['Data_Value']=matches_2020['Data_Value'].astype(int)
	#print(matches_2020['StateAbbr'].unique())
	xbar = len(matches_2020['Measure'].unique()) # match all of the unique values
	long_dataframe_list=[]
	for k in range(xbar):
	
		first_row = List_of_min_max.iloc[k] # Matches the same index in my new df because they should be the same
		#print(first_row)
		state_abbr = first_row[0]
		#print(state_abbr)
		measure = first_row[3]
		#print(measure)
		maxes1 = first_row[2].astype(int)
		#print(maxes1)
		mins1 = first_row[1].astype(int)
		#print(mins1)
		condition = matches_2020[matches_2020['Measure']== measure]
		filtering_now = condition[condition['Data_Value'] >= mins1]
		filtering_now1 = filtering_now[filtering_now['Data_Value'] <= maxes1]
		#result_df = pd.concat([result_df,filtering_now1])
		result_df1= filtering_now1.values.tolist()
		long_dataframe_list.extend(result_df1)
		
	results_test=pd.DataFrame(long_dataframe_list,columns=column_names)
	dataframes_list2020_1.append(results_test) # Remember to change this after debugging


In [ ]:
dataframes_list2020_1[1].loc[dataframes_list2020_1[1]['Measure']=='Current_smoking_among_adults_aged_>=18_years','Data_Value'].describe()

Getting rid of outliers for 2021

In [ ]:
x_sum1=[]
y=[]
n = len(dataframes_list2021)

new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2021 = dataframes_list2021[j] # matches 2020 is first dataframe
	subseta=(matches_2021['StateAbbr'].unique()) # Just a list of the state names in that df
	#print(subseta)
	matching_indices1 = [index for index, df in enumerate(dataframes_list2021) if (df['StateAbbr'].isin(subseta).any())]# No matter where they are, this could should find the matching states. 
	for index in matching_indices1:
		matches_2021 = dataframes_list2021[index]
		subset1=(matches_2021['Measure'].unique()) 
		intersection_list = set(list(subset1))
		for i in intersection_list:
			lis1 = matches_2021.loc[matches_2021['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
			#print(len(lis1))
			my_array = np.array(lis1)
			q1,q3 = np.percentile(my_array, (25,75))
			iqr1 = q3-q1		
			#print(iqr)
			min1 = q1-(1.5*iqr1) # Subtract from Q1
			#(print(min1))
			max1 = (1.5*iqr1)+q3 # Add to Q3
			x_sum1.append([subseta,min1,max1,i,q1,q3])
			#print("Total prints:", print_count)

In [ ]:
List_of_min_max=pd.DataFrame(x_sum1) # Put those in a dataframe
List_of_min_max['StateAbbr'] = List_of_min_max[0].astype(str)
List_of_min_max['StateAbbr'] = List_of_min_max['StateAbbr'].str.replace("['", '').str.replace("']", '')
List_of_min_max["StateAbbr"] = List_of_min_max["StateAbbr"].str.strip()
List_of_min_max['min1'] = List_of_min_max[1].astype(float)
List_of_min_max['max1'] = List_of_min_max[2].astype(float)
List_of_min_max['Measure'] = List_of_min_max[3].astype(str)
List_of_min_max['Measure'] = List_of_min_max['Measure'].str.strip()
List_of_min_max['Q1'] = List_of_min_max[4].astype(float)
List_of_min_max['Q3'] = List_of_min_max[5].astype(float)
List_of_min_max= List_of_min_max.drop([0,1,2,3,4,5],axis=1)
List_of_min_max_2021 =List_of_min_max

In [ ]:
column_names=s2021.columns.values.tolist()
dataframes_list2021_1 =[] #<- New Dataframe list
n = len(dataframes_list2021)

result_df = pd.DataFrame()
for j in range(n):

	matches_2021 = dataframes_list2021[j] # <- Change this matches 2020 is first dataframe
	matches_2021['Measure'] = matches_2021['Measure'].str.strip()
	matches_2021['StateAbbr']=matches_2021['StateAbbr'].str.strip()
	matches_2021['Data_Value']=matches_2021['Data_Value'].astype(int)
	#print(matches_2020['StateAbbr'].unique())
	xbar = len(matches_2021['Measure'].unique()) # match all of the unique values
	long_dataframe_list=[]
	for k in range(xbar):
	
		first_row = List_of_min_max_2021.iloc[k] # Matches the same index in my new df because they should be the same
		#print(first_row)
		state_abbr = first_row[0]
		#print(state_abbr)
		measure = first_row[3]
		#print(measure)
		maxes1 = first_row[2].astype(int)
		#print(maxes1)
		mins1 = first_row[1].astype(int)
		#print(mins1)
		condition = matches_2021[matches_2021['Measure']== measure]
		filtering_now = condition[condition['Data_Value'] >= mins1]
		filtering_now1 = filtering_now[filtering_now['Data_Value'] <= maxes1]
		#result_df = pd.concat([result_df,filtering_now1])
		result_df1= filtering_now1.values.tolist()
		long_dataframe_list.extend(result_df1)
		
	results_test=pd.DataFrame(long_dataframe_list,columns=column_names)
	dataframes_list2021_1.append(results_test) # Remember to change this after debugging

In [ ]:
dataframes_list2021_1[0]['Measure'].nunique()
dataframes_list2021_1[0].loc[dataframes_list2021_1[0]['Measure']=='Current_smoking_among_adults_aged_>=18_years','Data_Value'].describe()

In [ ]:
dataframes_list2021[0].loc[dataframes_list2021[0]['Measure']=='Current_smoking_among_adults_aged_>=18_years','Data_Value'].describe()

Getting rid of outliers 2022

In [ ]:
x_sum2=[]
y=[]
n = len(dataframes_list2022)

new_list_of_measures=[]
print_count = 0
for j in range(n):
	matches_2022 = dataframes_list2022[j] # matches 2020 is first dataframe
	subseta=(matches_2022['StateAbbr'].unique()) # Just a list of the state names in that df
	#print(subseta)
	matching_indices1 = [index for index, df in enumerate(dataframes_list2022) if (df['StateAbbr'].isin(subseta).any())]# No matter where they are, this could should find the matching states. 
	for index in matching_indices1:
		matches_2022 = dataframes_list2022[index]
		subset1=(matches_2022['Measure'].unique()) 
		intersection_list = set(list(subset1))
		for i in intersection_list:
			lis1 = matches_2022.loc[matches_2022['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
			#print(len(lis1))
			my_array = np.array(lis1)
			q1,q3 = np.percentile(my_array, (25,75))
			iqr1 = q3-q1		
			#print(iqr)
			min1 = q1-(1.5*iqr1) # Subtract from Q1
			#(print(min1))
			max1 = (1.5*iqr1)+q3 # Add to Q3
			x_sum2.append([subseta,min1,max1,i,q1,q3])
			#print("Total prints:", print_count)

In [ ]:
List_of_min_max=pd.DataFrame(x_sum2) # Put those in a dataframe
List_of_min_max['StateAbbr'] = List_of_min_max[0].astype(str)
List_of_min_max['StateAbbr'] = List_of_min_max['StateAbbr'].str.replace("['", '').str.replace("']", '')
List_of_min_max["StateAbbr"] = List_of_min_max["StateAbbr"].str.strip()
List_of_min_max['min1'] = List_of_min_max[1].astype(float)
List_of_min_max['max1'] = List_of_min_max[2].astype(float)
List_of_min_max['Measure'] = List_of_min_max[3].astype(str)
List_of_min_max['Measure'] = List_of_min_max['Measure'].str.strip()
List_of_min_max['Q1'] = List_of_min_max[4].astype(float)
List_of_min_max['Q3'] = List_of_min_max[5].astype(float)
List_of_min_max= List_of_min_max.drop([0,1,2,3,4,5],axis=1)
List_of_min_max_2022 =List_of_min_max

In [ ]:
column_names=s2022.columns.values.tolist()
dataframes_list2022_1 =[] #<- New Dataframe list
n = len(dataframes_list2022)

result_df = pd.DataFrame()
for j in range(n):

	matches_2022 = dataframes_list2022[j] # <- Change this matches 2020 is first dataframe
	matches_2022['Measure'] = matches_2022['Measure'].str.strip()
	matches_2022['StateAbbr']=matches_2022['StateAbbr'].str.strip()
	matches_2022['Data_Value']=matches_2022['Data_Value'].astype(int)
	#print(matches_2020['StateAbbr'].unique())
	xbar = len(matches_2022['Measure'].unique()) # match all of the unique values
	long_dataframe_list=[]
	for k in range(xbar):
	
		first_row = List_of_min_max_2022.iloc[k] # Matches the same index in my new df because they should be the same
		#print(first_row)
		state_abbr = first_row[0]
		#print(state_abbr)
		measure = first_row[3]
		#print(measure)
		maxes1 = first_row[2].astype(int)
		#print(maxes1)
		mins1 = first_row[1].astype(int)
		#print(mins1)
		condition = matches_2022[matches_2022['Measure']== measure]
		filtering_now = condition[condition['Data_Value'] >= mins1]
		filtering_now1 = filtering_now[filtering_now['Data_Value'] <= maxes1]
		#result_df = pd.concat([result_df,filtering_now1])
		result_df1= filtering_now1.values.tolist()
		long_dataframe_list.extend(result_df1)
		
	results_test=pd.DataFrame(long_dataframe_list,columns=column_names)
	dataframes_list2022_1.append(results_test) # Remember to change this after debugging

In [ ]:
dataframes_list2022_1[0]['Measure'].nunique()
dataframes_list2022_1[0].loc[dataframes_list2022_1[0]['Measure']=='Current_smoking_among_adults_aged_>=18_years','Data_Value'].describe()

In [ ]:
dataframes_list2022[0].loc[dataframes_list2022[0]['Measure']=='Current_smoking_among_adults_aged_>=18_years','Data_Value'].describe()

In [ ]:
dataframes_list2022_1[50].isnull().sum()

All Cells seem normal. The values update accordingly.

In [ ]:
dataframes_list2020 = dataframes_list2020_1
dataframes_list2021 = dataframes_list2021_1
dataframes_list2022 = dataframes_list2022_1

# Okay Now time for Density Plots. 
Density plots can be very useful but with huge amounts of data can be overwhelming. 

The below code visualizes just a few. 
In order to plot all of them would require a huge amount of computing power. So it does not make sense to do this for so much data.

In [ ]:

lmn = list(s2020['Measure'].unique())


In [ ]:
lmn[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

n = 5
lmn = list(s2020['Measure'].unique())

plt.figure(figsize=(10, 6))
abc_index = 0  # Initialize index for lmn
for j in range(n):
    dataframe_needed = dataframes_list2020_1[j]
    abc = lmn[abc_index]
    data_time = dataframe_needed.loc[dataframe_needed['Measure'] == abc, 'Data_Value']
    sns.kdeplot(data=data_time, fill=True, label=f'DataFrame {j + 1} - {dataframe_needed["StateAbbr"].unique()}')

    if j == n:
        # Increment index for abc if it's the last dataframe
        abc_index += 1

plt.legend(title='DataFrame')
plt.title(f'{abc}')
plt.xlabel('Data Value')
plt.show()
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

n = 10
lmn = list(s2020['Measure'].unique())

plt.figure(figsize=(10, 6))
abc_index = 0  # Initialize index for lmn
for j in range(5,n):
    dataframe_needed = dataframes_list2020_1[j]
    abc = lmn[abc_index]
    data_time = dataframe_needed.loc[dataframe_needed['Measure'] == abc, 'Data_Value']
    sns.kdeplot(data=data_time, fill=True, label=f'DataFrame {j + 1} - {dataframe_needed["StateAbbr"].unique()}')

    if j == n:
        # Increment index for abc if it's the last dataframe
        abc_index += 1

plt.legend(title='DataFrame')
plt.title(f'{abc}')
plt.xlabel('Data Value')
plt.show()
n = 15
lmn = list(s2020['Measure'].unique())

plt.figure(figsize=(10, 6))
abc_index = 0  # Initialize index for lmn
for j in range(10,n):
    dataframe_needed = dataframes_list2020_1[j]
    abc = lmn[abc_index]
    data_time = dataframe_needed.loc[dataframe_needed['Measure'] == abc, 'Data_Value']
    sns.kdeplot(data=data_time, fill=True, label=f'DataFrame {j + 1} - {dataframe_needed["StateAbbr"].unique()}')

    if j == n:
        # Increment index for abc if it's the last dataframe
        abc_index += 1

plt.legend(title='DataFrame')
plt.title(f'{abc}')
plt.xlabel('Data Value')
plt.show()


In [ ]:
dataframes_list2020_1[0][dataframes_list2020_1[0]['Measure']=="Taking_medicine_for_high_blood_pressure_control_among_adults_aged_>=18_years_with_high_blood_pressure"].describe()

In [ ]:
s2020[s2020['Measure']=="Taking_medicine_for_high_blood_pressure_control_among_adults_aged_>=18_years_with_high_blood_pressure"].describe()

# So what to do when visualization is unrealistic but we need to check out the distributions of the data so the right type of test can be applied?

Some python functions help us out here. 
distfit is a great one for example. It can determine what the best distribution fit would be for data \
This was already performed elsewhere and the results are loaded in below. 

https://pypi.org/project/distfit/



In [ ]:

normality_2021 = pd.read_csv('2021_data_df')
normality_2022 = pd.read_csv('2022_data_df')
normality_2020 = pd.read_csv('2020_data_df')
print(('normal',len(normality_2020[normality_2020['Best_Fit_Distribution']=='norm'])),
('not_normal',len(normality_2020[normality_2020['Best_Fit_Distribution']!='norm'])),
('normal',len(normality_2021[normality_2021['Best_Fit_Distribution']=='norm'])),
('not_normal',len(normality_2021[normality_2021['Best_Fit_Distribution']!='norm'])),
('normal',len(normality_2022[normality_2022['Best_Fit_Distribution']=='norm'])),
('not_normal',len(normality_2022[normality_2022['Best_Fit_Distribution']!='norm'])),sep='\n')

## Since the data are overwhelmingly non-normally distributed.

We can use the Spearman's rank-order correlation.
Which is the non-parametric version of the pearson product moment correlation test. \
It helps show the strength of the relationship between two variables. 
With a -1 or a +1 denoting perfect positive or negative relationship. \
So what is needed for this analysis are all of the values that are as close to 0 as possible.


## The formula for the Spearman's rank order correlation:

$$ r_s = \rho R(X),R(Y)= \frac{cov(R(X),R(Y))}{\sigma_{R(X)} \sigma_{R(Y)}} $$


$$cov(R(X),R(Y)) = {\text{The covariance of the rank variables}}$$ 

and p denotes the pearson correlation coefficient.

And once all of the values are obtained, the data can be fiiltered leaving only uncorrelated pairs of variables.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html


In [ ]:




def find_my_state(states,dataframe_list):
	state = []
	x =[]
	y=[]
	n=len(dataframe_list)
	for state in states:
		state = set(state['StateAbbr'].unique())  # Get the unique state name
		for i in range(n):
			matches_2020 = dataframe_list[i]
			subseta=set(matches_2020['StateAbbr'].unique())
			if state == subseta:
				matches_texas = dataframe_list[i]
				subset1 = (matches_texas['Measure'].unique())
				subsetb=(matches_texas['StateAbbr'].unique())
			else:
				i +=1
		for i in range(n):
			not_texas = dataframe_list[i] # Otherwise iterate over everything
			subsetc = (not_texas['StateAbbr'].unique())
			subset2 = (not_texas['Measure'].unique())
			
			intersection_of_measures = set(subset1).intersection(set(subset2)) # now the intersection of all 2	
			intersection_list = list(intersection_of_measures)

			for i in intersection_list:
				y.append([i])
				lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
				lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
				if len(lis1) != len(lis2):
					min_len = (min(len(lis1), len(lis2)))
					
					if len(lis1) == min_len:
				
						lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement and also makes sure the shorter list length is used
						
					if len(lis2) == min_len:
						lis1= random.sample(list(lis1), min_len)
					else:
						continue

					x.append([subsetb,subsetc,stats.spearmanr(lis1, lis2),i])
	return x


The below code takes a list of the dataframes, and the function. And outputs the filtered p values and such.

In [ ]:
list_of_states = [TX,CA,PA]
yay=find_my_state(list_of_states,dataframes_list2020)

In [ ]:
list_of_states = [TX,CA,PA]
yay1=find_my_state(list_of_states,dataframes_list2021)

In [ ]:
list_of_states = [TX,CA,PA]
yay2=find_my_state(list_of_states,dataframes_list2022)

In [ ]:
# This code makes the needed df for all specific variables. And cleans up. And filters. 
df1=pd.DataFrame()
def now_make_dataframes(xm):
	df1 = pd.DataFrame(xm)
	df1['state'] = df1[0]+df1[1]
	df1['state'] = df1['state'].astype(str)
	df1['result_string'] = df1[2].astype(str) # gotta change the kurkwalis to string
	df1['Corr'] = df1['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
	df1['p-value'] = df1['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
	df1['State1'] = df1[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ','.join(x[0]))
	df1['State2'] = df1[1].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ','.join(x[0]))
	df1['p-value'] = df1['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
	df1['p-value'] = pd.to_numeric(df1['p-value'], errors='coerce') # Convert P value to numeric
	df1['Measure'] = df1[3]
	df1= df1.drop(['result_string',0,1,2,3],axis=1)
	empty_rows = df1[df1.isna().any(axis=1)]
	df1 = df1[~df1.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
	filtered_df = df1[df1['Corr'] < 0.5]
	filtered_df1 = filtered_df[filtered_df["Corr"]>-0.5]
	return filtered_df1


In [ ]:
filtered_df1=now_make_dataframes(yay)

In [ ]:
filtered_df1.head(1)

In [ ]:
filtered_df2=now_make_dataframes(yay1)

In [ ]:
filtered_df2.head(1)

In [ ]:
filtered_df3=now_make_dataframes(yay2)

In [ ]:
filtered_df3.head(1)

Checking below if there are any duplicates

In [ ]:
duplicates10 = filtered_df1[filtered_df1.duplicated(['Measure','state'])]

In [ ]:
duplicates10

In [ ]:
duplicates10 = filtered_df2[filtered_df2.duplicated(['Measure','state'])]
duplicates10

In [ ]:
duplicates10 = filtered_df3[filtered_df3.duplicated(['Measure','state'])]
duplicates10

No duplicates is a good sign. 

# So this Value below can be used to select all of the state combinations that pass correlation test and visualize them

In [ ]:
grouped = filtered_df1.groupby('State2') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')


In [ ]:
grouped = filtered_df2.groupby('State2') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')

In [ ]:
grouped = filtered_df3.groupby('State2') # So group each by state
fig, axes = plt.subplots(nrows=11, ncols=5, figsize=(50, 55))  # Adjust figsize as needed

for i, (state, group) in enumerate(grouped):
    ax = axes[i // 5, i % 5]  # Calculate the correct subplot based on row and column ( a 10 by 5 basically with extra steps)
    ax.hist(group['Corr'], bins=20, edgecolor='k', alpha=0.7)
    ax.set_title(f'Distribution of Corr Values for {state}')
    ax.set_xlabel('Corr Value')
    ax.set_ylabel('Frequency')

# So given the original question: Do states with more places data have different metrics in general?

Below are the counts for all of the places data divided by state, PA, TX, CA, IL, have the most by a wide margin so let's look at those first.

In [ ]:
m=s2020[s2020['StateAbbr']=='NC']
m['LocationName'].nunique()

In [ ]:
# Data
years = ['2020', '2021', '2022']
values = [28329, 28329, 28329]


plt.bar(years, values, color='green')

plt.xlabel('Year')
plt.ylabel('Values')
plt.title('Sum of all cities in PLACES Dataset per year')


plt.show()

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].nunique()
city_counts1 = city_counts1.sort_values(ascending=False)
city_counts2 = s2021.groupby('StateAbbr')['LocationName'].nunique()
city_counts2 = city_counts2.sort_values(ascending=False)
city_counts3 = s2022.groupby('StateAbbr')['LocationName'].nunique()
city_counts3 = city_counts3.sort_values(ascending=False)

In [ ]:
city_counts1['CA']

In [ ]:
city_counts1['PA']

In [ ]:
city_counts1['TX']

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts1,palette="dark:#5A9_r")


ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts2,palette="dark:#5A9_r")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2021")

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts3,palette="dark:#5A9_r")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2022")

## Kruskal-Wallis H Test 

The Kruskal-Wallis H-test tests whether the null hypothesis that the population median of all of groups are equal. It is a non-parametric version of ANOVA. The test works on 2 or more independent samples, which may have different sizes. Note that rejecting the null hypothesis does not indicate which of the groups differs. Post hoc comparisons between groups are required to determine which groups are different.*

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html


For the code below. I take each state index, match the same states in the other dataframes and then run all stats kruskal tests on the variables. This test can also take in arrays of different lengths so I commented out the random sampling to make them the same. 

I need to see if the medians of the groups are rejected more for the cities with more places data than others. Over years. 

In [ ]:
filtered_df1.head(1)

In [ ]:
def run_kruskal(states,dataframe_list):
	state = []
	x =[]
	y=[]
	n=len(dataframe_list)
	for state in states:
		state = set(state['StateAbbr'].unique())  # Get the unique state name
		for i in range(n):
			matches_2020 = dataframe_list[i]
			subseta=set(matches_2020['StateAbbr'].unique())
			if state == subseta:
				matches_texas = dataframe_list[i]
				subset1 = (matches_texas['Measure'].unique())
				subsetb=(matches_texas['StateAbbr'].unique())
			else:
				i +=1
		for i in range(n):
			not_texas = dataframe_list[i] # Otherwise iterate over everything
			subsetc = (not_texas['StateAbbr'].unique())
			subset2 = (not_texas['Measure'].unique())
			
			intersection_of_measures = set(subset1).intersection(set(subset2)) # now the intersection of all 2	
			intersection_list = list(intersection_of_measures)

			for i in intersection_list:
				y.append([i])
				lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
				lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
				#if len(lis1) != len(lis2):
					#min_len = (min(len(lis1), len(lis2)))
					
					#if len(lis1) == min_len:
				
						#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement and also makes sure the shorter list length is used
						
					#if len(lis2) == min_len:
						#lis1= random.sample(list(lis1), min_len)
					#else:
						#continue

				x.append([subsetb,subsetc,stats.stats.kruskal(lis1, lis2),i])
	return x

In [ ]:
states_list_1 = [TX,PA,CA]
yay5=run_kruskal(states_list_1,dataframes_list2020)

In [ ]:
states_list_2 = [TX,PA,CA]
yay6=run_kruskal(states_list_2,dataframes_list2021)

In [ ]:
states_list_3 = [TX,PA,CA]
yay7=run_kruskal(states_list_3,dataframes_list2022)

In [ ]:
yay5[0]

In [ ]:
# This code makes the needed df for all specific variables. And cleans up. And filters. 
df1=pd.DataFrame()
def now_make_dataframes(xm):
	df1 = pd.DataFrame(xm)
	df1['state'] = df1[0]+df1[1]
	df1['state'] = df1['state'].astype(str)
	df1['result_string'] = df1[2].astype(str) # gotta change the kurkwalis to string
	df1['Statistic'] = df1['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
	df1['p-value'] = df1['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
	df1['State1'] = df1[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ','.join(x[0]))
	df1['State2'] = df1[1].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ','.join(x[0]))
	df1['p-value'] = df1['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
	df1['p-value'] = pd.to_numeric(df1['p-value'], errors='coerce') # Convert P value to numeric
	df1['Measure'] = df1[3]
	df1= df1.drop(['result_string',0,1,2,3],axis=1)
	empty_rows = df1[df1.isna().any(axis=1)]
	df1 = df1[~df1.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
	return df1


In [ ]:
filtered_df_kruskal_2020=now_make_dataframes(yay5)

In [ ]:
filtered_df_kruskal_2021=now_make_dataframes(yay6)

In [ ]:
filtered_df_kruskal_2022=now_make_dataframes(yay7)

In [ ]:
shorted_df1 = filtered_df1.drop(columns=['Corr','p-value','State1','State2'],axis=1)
shorted_df2 = filtered_df2.drop(columns=['Corr','p-value','State1','State2'],axis=1)
shorted_df3 = filtered_df3.drop(columns=['Corr','p-value','State1','State2'],axis=1)

In [ ]:
shorted_df1.head(1)

In [ ]:
result_2020_kruskal = filtered_df_kruskal_2020.merge(shorted_df1, how='inner', on=['Measure','state'])
len(result_2020_kruskal)

In [ ]:
result_2021_kruskal = filtered_df_kruskal_2021.merge(shorted_df2, how='inner', on=['Measure','state'])
len(result_2021_kruskal)

In [ ]:
result_2022_kruskal = filtered_df_kruskal_2022.merge(shorted_df3, how='inner', on=['Measure','state'])
len(result_2022_kruskal)

In [ ]:
filtered_df3.head(1)

# 11/10/2023
So to see if Texas for example differed more than any other state, I need to sum all of the times texas occurs and then sum all of the other states. The above filtered DF is where there is a difference from the median. And if texas occurs more, then.. it occurs more. It does not answer the question on if the data is higher or lower. Just that a difference exists.

In [ ]:
from collections import Counter
print(Counter(filtered_df1['State1']))

In [ ]:
from collections import Counter
print(Counter(filtered_df1['State2']))

In [ ]:
from collections import Counter
print(Counter(filtered_df2['State1']))

In [ ]:
from collections import Counter
print(sum(Counter(filtered_df2['State2'])))

Since all of the results are overwhelmingly, yes the values are different. Post Hoc analysis must be performed.The dunn

# Start of the Dunn test 

Since the data are mostly nonparametric. I can only say if they are different from each other, and not whether one is higher or lower. The posthoc Dunn test performs this analysis. 


In [ ]:
filtered_df_kruskal_2020.head(1)

In [ ]:
ugh = [] # New  Empty List
state_names = []

list_of_measures1 = s2020['Measure'].unique().tolist() # Put all unique values in a list
for i in list_of_measures1:

	result1 = s2020[s2020['Measure'] == 'Cancer_(excluding_skin_cancer)_among_adults_aged_>=18_years'] # Go over each measure
	for i in new_list_of_titles:
		condition = lambda x: x['StateAbbr'] == i  # Gather all of the state data
		filtered_data = result1[result1.apply(condition, axis=1)] # Filter all of the state data
		lis1 = filtered_data['Data_Value'].tolist()
		ugh.append(lis1)
		state_names.append(i)
	ab= sp.posthoc_dunn(ugh, p_adjust = 'bonferroni')
	df_dunn_result = pd.DataFrame(ab, index=ab.index, columns=ab.columns) # in order to get the data to appear appropriately, need to include index and columns
	filtered_df5=df_dunn_result.where(df_dunn_result<0.05)
	indices = [(l, m) for l, row in enumerate(filtered_df5.values) for m, value in enumerate(row) if pd.notna(value)]

In [ ]:
state_mapping = {index: state_names[i] for i, index in enumerate(range(len(state_names)))} # Enumerates the state names list
state_pairs = [(state_mapping[i], state_mapping[j]) for i, j in indices] # Maps the state names listto its respective index. 
sorting_pairs = set([tuple(sorted(pair)) for pair in state_pairs]) # Okay so sort the things and make them into a set. 

In [ ]:
only_dunn_TX=(df_dunn_result[43].where(df_dunn_result[43]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_TX=only_dunn_TX.dropna()
len(only_dunn_TX)
only_dunn_CA=(df_dunn_result[4].where(df_dunn_result[4]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_CA=only_dunn_CA.dropna()
len(only_dunn_CA)
only_dunn_PA=(df_dunn_result[38].where(df_dunn_result[38]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_PA=only_dunn_PA.dropna()
len(only_dunn_PA)
only_dunn_IL=(df_dunn_result[12].where(df_dunn_result[12]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_IL=only_dunn_IL.dropna()
len(only_dunn_IL)
only_dunn_SD=(df_dunn_result[41].where(df_dunn_result[41]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_SD=only_dunn_SD.dropna()
len(only_dunn_SD)
only_dunn_WY=(df_dunn_result[50].where(df_dunn_result[50]<0.05)) # Now we have the Results for this one column. And I will sort by p value less than 0.05
only_dunn_WY=only_dunn_WY.dropna()
len(only_dunn_WY)

In [ ]:
len(only_dunn_TX)

#11/11/2023 Sorted all of the state counts
So using the above code and visualizing it. It is easy to see that Over all Texas

In [ ]:
print(len(set(indices)))

In [ ]:
# Have to count rows and columns separately. 
rows, columns = zip(*indices)

# Count occurrences of each row and column index
row_counts = Counter(rows)
column_counts = Counter(columns)
dataframe_for_rows=pd.DataFrame.from_dict(row_counts, orient='index', columns=['Count'])
dataframe_for_rows['States']= [state_names[row] for row in dataframe_for_rows.index]

In [ ]:
dataframe_for_rows1=pd.DataFrame.from_dict(row_counts, orient='index', columns=['Count'])
# Now I have to calculate the weights


In [ ]:
dataframe_for_rows2 = dataframe_for_rows.set_index(['States'], drop=True)

In [ ]:
hm=dataframe_for_rows2.T
print(hm.to_markdown())

In [ ]:
dataframe_for_rows2['calculated'] = dataframe_for_rows2['Count'].apply(lambda x: 'yes' if x < 46 else 'no')

In [ ]:
len(dataframe_for_rows2[dataframe_for_rows2['calculated']=='yes'])

In [ ]:


fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(hm,palette="cividis")

ax.set(xlabel="States",ylabel ="Count ",title="Locations per State 2020")

# It seems a parametric set of tests will be required after all. 
But how can parametric tests be used on this data?

Well. I can tell that Texas is often different from the state to which it is compared. But which direction it differs is hard to discern.So maybe it is time to employ some parametric methods after all.

With large enough sample sizes (> 30 or 40), the violation of the normality assumption should not cause major problems * \
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3693611/*

And each sample size is a few hundred. So all measures and States satisfy this rule. 


# LEVENE Test
Going to use the levene test to see if samples come from equal variance distribution
If the resulting p-value of Levene's test is less than some significance level (typically 0.05), \
 the obtained differences in sample variances are unlikely to have occurred based on random sampling from a \
population with equal variances. Thus, the null hypothesis of equal variances is rejected and it is concluded that there is a difference \
 between the variances in the population. 

The scipy stats Levene test library

https://en.wikipedia.org/wiki/Levene%27s_test

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html

In [ ]:
prints=filtered_df1[['Measure','State1','State2']]
n = len(prints)
current = prints.values.tolist()

In [ ]:
def run_levenes(states,dataframe_list):
	state = []
	x =[]
	y=[]
	n=len(dataframe_list)
	for state in states:
		state = set(state['StateAbbr'].unique())  # Get the unique state name
		for i in range(n):
			matches_2020 = dataframe_list[i]
			subseta=set(matches_2020['StateAbbr'].unique())
			if state == subseta:
				matches_texas = dataframe_list[i]
				subset1 = (matches_texas['Measure'].unique())
				subsetb=(matches_texas['StateAbbr'].unique())
			else:
				i +=1
		for i in range(n):
			not_texas = dataframe_list[i] # Otherwise iterate over everything
			subsetc = (not_texas['StateAbbr'].unique())
			subset2 = (not_texas['Measure'].unique())
			
			intersection_of_measures = set(subset1).intersection(set(subset2)) # now the intersection of all 2	
			intersection_list = list(intersection_of_measures)

			for i in intersection_list:
				y.append([i])
				lis1 = matches_texas.loc[matches_texas['Measure']==i,'Data_Value'] # select all of the values in the dataframe that match
				lis2 = not_texas.loc[not_texas['Measure']==i,'Data_Value']
				#if len(lis1) != len(lis2):
					#min_len = (min(len(lis1), len(lis2)))
					
					#if len(lis1) == min_len:
				
						#lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement and also makes sure the shorter list length is used
						
					#if len(lis2) == min_len:
						#lis1= random.sample(list(lis1), min_len)
					#else:
						#continue

				x.append([subsetb,subsetc,stats.levene(lis1, lis2),i])
	return x


In [ ]:
list_for_levene = [TX,PA,CA]
trying_2020 = run_levenes(list_for_levene, dataframes_list2020)
trying_2021 = run_levenes(list_for_levene, dataframes_list2021)
trying_2022 = run_levenes(list_for_levene, dataframes_list2022)

In [ ]:
df1=pd.DataFrame()
def now_make_dataframes(xm):
	df1 = pd.DataFrame(xm)
	df1['state'] = df1[0]+df1[1]
	df1['state'] = df1['state'].astype(str)
	df1['result_string'] = df1[2].astype(str) # gotta change the kurkwalis to string
	df1['Statistic'] = df1['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
	df1['p-value'] = df1['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
	df1['State1'] = df1[0].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ','.join(x[0]))
	df1['State2'] = df1[1].astype(str).str.extractall(r'\'([A-Z]{2})\'').groupby(level=0).apply(lambda x: ','.join(x[0]))
	df1['p-value'] = df1['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
	df1['p-value'] = pd.to_numeric(df1['p-value'], errors='coerce') # Convert P value to numeric
	df1['Measure'] = df1[3]
	df1= df1.drop(['result_string',0,1,2,3],axis=1)
	empty_rows = df1[df1.isna().any(axis=1)]
	df1 = df1[~df1.index.isin(empty_rows.index)] # Use boolean indexing to get rid of all of the empty values 
	return df1

In [ ]:
levene_attempt2020 = now_make_dataframes(trying_2020)
levene_attempt2021 = now_make_dataframes(trying_2021)
levene_attempt2022 = now_make_dataframes(trying_2022)

In [ ]:
levene_attempt2021.head(2)

In [ ]:
shorted_df1 = filtered_df1.drop(columns=['Corr','p-value','State1','State2'],axis=1)
shorted_df2 = filtered_df2.drop(columns=['Corr','p-value','State1','State2'],axis=1)
shorted_df3 = filtered_df3.drop(columns=['Corr','p-value','State1','State2'],axis=1)

In [ ]:
result_2020_levene = levene_attempt2020.merge(shorted_df1, how='right', on=['Measure','state'])
result_2021_levene = levene_attempt2021.merge(shorted_df2, how='right', on=['Measure','state'])
result_2022_levene = levene_attempt2022.merge(shorted_df3, how='right', on=['Measure','state'])

In [ ]:
len(result_2020_levene)

In [ ]:
len(result_2021_levene)

In [ ]:
len(result_2022_levene)

# So we have the results of the Levene test for 2020 for texas above
Which will tell us if the values come from equal or unequal variances.


# T test starts Below

In [ ]:
abcd = result_2020_levene[result_2020_levene['p-value'] < 0.05] # For levenes test lower p values mean unequal variance
defg=result_2020_levene[result_2020_levene['p-value'] >0.05 ] #Equal variance
prints=defg[['Measure','State1','State2']]
n = len(prints)
current = prints.values.tolist()
#Levene_test_2020 =defg.groupby('state')['Measure1'].nunique()#
#unequal_variances = df2[df2.index.isin(defg.index)]#

In [ ]:

def running_t_equal_var(levene_results,dataframe):
	x=[]
	y=[]
	defg=[]
	prints=[]
	current=[]
	defg=levene_results[levene_results['p-value'] >0.05 ]
	prints=defg[['Measure','State1','State2']]
	n = len(prints)
	current = prints.values.tolist()
	for i in range(n):  # For i in range n, set the current list parts to separate variables.
		current1 = current[i]
		measure=(current1[0])

		state1=(current1[1]) # first second, third element
		state2=(current1[2].strip()) # Need to strip random whitesapce


		condition1 = dataframe.loc[dataframe['StateAbbr']==state1]
		#print(len(condition1))
		lis1 = condition1.loc[condition1['Measure']==measure,'Data_Value'] # For a dataframe .loc seems to be the simplest for performing this work
		condition2 = dataframe.loc[dataframe['StateAbbr']==state2]
		#print(len(condition2))
		lis2 = condition2.loc[condition2['Measure']==measure,'Data_Value']
		y.append([state1,state2])
		if len(lis1) != len(lis2):
			min_len = (min(len(lis1), len(lis2)))
			
			if len(lis1) == min_len:

				lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				
			if len(lis2) == min_len:
				lis1= random.sample(list(lis1), min_len)
		else:
			continue
			#print(len(lis1),len(lis2))
		#print(stats.ttest_ind(lis1, lis2,equal_var=True,alternative='greater'),i)
		x.append([state1,state2,measure,stats.ttest_ind(lis1, lis2,equal_var=True,alternative='greater'),i]) # Make sure to set the alternative to greater
		#print("Total prints:", print_count)
		#result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together
	return x

In [ ]:
trying_t_test_2020=running_t_equal_var(result_2020_levene,s2020)
trying_t_test_2021=running_t_equal_var(result_2021_levene,s2021)
trying_t_test_2022=running_t_equal_var(result_2022_levene,s2022)

In [ ]:
df1=pd.DataFrame()
def now_make_dataframes1(xm):
	df1=pd.DataFrame(xm)
	df1['state'] = df1[0]+df1[1]
	df1['state'] = df1['state'].astype(str)
	df1['result_string'] = df1[3].astype(str) # gotta change the kurkwalis to string
	df1['Statistic'] = df1['result_string'].str.extract(r'statistic=([0-9.-]+)').astype(float) # Then just regex it away
	df1['p-value'] = df1['result_string'].str.extract(r'pvalue=([0-9.eE+-]+)').astype(float)
	df1['State1'] = df1[0].astype(str)
	df1['State2'] = df1[1].astype(str)
	df1['p-value'] = df1['p-value'].astype(float).apply(lambda x: '{:.10f}'.format(x) if pd.notna(x) else '')
	df1['p-value'] = pd.to_numeric(df1['p-value'], errors='coerce') # Convert P value to numeric
	df1['Measure'] = df1[2].astype(str)
	df1['Measure'] = df1['Measure'].str.strip()
	df1= df1.drop(['result_string',0,1,2,3,4],axis=1)
	empty_rows = df1[df1.isna().any(axis=1)]
	df1 = df1[~df1.index.isin(empty_rows.index)]
	return df1

In [ ]:
t_test_equal_var_2020=now_make_dataframes1(trying_t_test_2020)
t_test_equal_var_2021=now_make_dataframes1(trying_t_test_2021)
t_test_equal_var_2022=now_make_dataframes1(trying_t_test_2022)

In [ ]:
t_test_equal_var_2020.head(1)

# T test unequal Var

In [ ]:
def running_t_unequal_var(levene_results,dataframe):
	x=[]
	y=[]
	defg=[]
	prints=[]
	defg=levene_results[levene_results['p-value'] <0.05 ]
	prints=defg[['Measure','State1','State2']]
	n = len(prints)
	current = prints.values.tolist()
	for i in range(n):  # For i in range n, set the current list parts to separate variables.
		current1 = current[i]
		measure=(current1[0])

		state1=(current1[1]) # first second, third element
		state2=(current1[2].strip()) # Need to strip random whitesapce


		condition1 = dataframe.loc[dataframe['StateAbbr']==state1]
		#print(len(condition1))
		lis1 = condition1.loc[condition1['Measure']==measure,'Data_Value'] # For a dataframe .loc seems to be the simplest for performing this work
		condition2 = dataframe.loc[dataframe['StateAbbr']==state2]
		#print(len(condition2))
		lis2 = condition2.loc[condition2['Measure']==measure,'Data_Value']
		y.append([state1,state2])
		if len(lis1) != len(lis2):
			min_len = (min(len(lis1), len(lis2)))
			
			if len(lis1) == min_len:

				lis2= random.sample(list(lis2), min_len) # Random sample does not use replacement
				
			if len(lis2) == min_len:
				lis1= random.sample(list(lis1), min_len)
		else:
			continue
			#print(len(lis1),len(lis2))
		#print(stats.ttest_ind(lis1, lis2,equal_var=True,alternative='greater'),i)
		x.append([state1,state2,measure,stats.ttest_ind(lis1, lis2,equal_var=False,alternative='greater'),i]) # Make sure to set the alternative to greater
		#print("Total prints:", print_count)
		#result1 = list(list(t) for t in zip(x, y)) # now just zip all of the lists together
	return x

In [ ]:
trying_t_test_2020_u=running_t_equal_var(result_2020_levene,s2020)
trying_t_test_2021_u=running_t_equal_var(result_2021_levene,s2021)
trying_t_test_2022_u=running_t_equal_var(result_2022_levene,s2022)

In [ ]:
t_test_unequal_var_2020=now_make_dataframes1(trying_t_test_2020_u)
t_test_unequal_var_2021=now_make_dataframes1(trying_t_test_2021_u)
t_test_unequal_var_2022=now_make_dataframes1(trying_t_test_2022_u)

In [ ]:
len(t_test_unequal_var_2020)


# Equal Var visualizations For Texas

In [ ]:
texas=t_test_equal_var_2020[t_test_equal_var_2020['State1']=='TX']
ca=t_test_equal_var_2020[t_test_equal_var_2020['State1']=='CA']
pa=t_test_equal_var_2020[t_test_equal_var_2020['State1']=='PA']

In [ ]:
prints=texas['State2']  # get the state all by itself
prints=prints.unique() 
lists1=[]
for i in prints:   
	abc = texas[texas['State2']==i]
	abcd = abc[abc['p-value'] <= 0.05]
	defg = abc[abc['p-value'] > 0.05]
	lists1.append(['TX',len(abcd),i,len(defg)])
	set_of_tuples = {tuple(inner_list) for inner_list in lists1}

In [ ]:
df10 = pd.DataFrame(set_of_tuples)
df10['combined'] = df10[0]+'/'+df10[2]


In [ ]:
X_1 = range(len(set_of_tuples))
x_val= [x[0] for x in set_of_tuples]
y_val = [x[1] for x in set_of_tuples]
x_val1 = [x[2] for x in set_of_tuples]
y_val1 =[x[3] for x in set_of_tuples]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(data=(y_val),alpha=0.9)
ax = sns.barplot(data=y_val1, alpha=0.7)
ax.set_xticklabels(df10['combined'], rotation='vertical', fontsize=10)
ax.set(xlabel='States',
       ylabel='count of p_values that are significant per state VS Texas',
       title='Resulting P values'
       )
ax.text(2, 9, "Texas = ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "blue",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(2.8, 10,"Other States= ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "orange",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(2.8, 11,"Texas Overlap = ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "brown",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 

In [ ]:
prints=ca['State2']  # get the state all by itself
prints=prints.unique() 
lists1=[]
for i in prints:   
	abc = ca[ca['State2']==i]
	abcd = abc[abc['p-value'] <= 0.05]
	defg = abc[abc['p-value'] > 0.05]
	lists1.append(['CA',len(abcd),i,len(defg)])
	set_of_tuples = {tuple(inner_list) for inner_list in lists1}

In [ ]:
df10 = pd.DataFrame(set_of_tuples)
df10['combined'] = df10[0]+'/'+df10[2]

X_1 = range(len(set_of_tuples))
x_val= [x[0] for x in set_of_tuples]
y_val = [x[1] for x in set_of_tuples]
x_val1 = [x[2] for x in set_of_tuples]
y_val1 =[x[3] for x in set_of_tuples]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(data=(y_val),alpha=0.9)
ax = sns.barplot(data=y_val1, alpha=0.7)
ax.set_xticklabels(df10['combined'], rotation='vertical', fontsize=10)
ax.set(xlabel='States',
       ylabel='count of p_values that are significant per state VS California',
       title='Resulting P values'
       )
ax.text(2, 9, "CA = ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "blue",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(2.8, 10,"Other States= ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "orange",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(2.8, 11,"CA Overlap = ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "brown",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 

In [ ]:
prints=pa['State2']  # get the state all by itself
prints=prints.unique() 
lists1=[]
for i in prints:   
	abc = pa[pa['State2']==i]
	abcd = abc[abc['p-value'] <= 0.05]
	defg = abc[abc['p-value'] > 0.05]
	lists1.append(['PA',len(abcd),i,len(defg)])
	set_of_tuples = {tuple(inner_list) for inner_list in lists1}

In [ ]:
df10 = pd.DataFrame(set_of_tuples)
df10['combined'] = df10[0]+'/'+df10[2]

X_1 = range(len(set_of_tuples))
x_val= [x[0] for x in set_of_tuples]
y_val = [x[1] for x in set_of_tuples]
x_val1 = [x[2] for x in set_of_tuples]
y_val1 =[x[3] for x in set_of_tuples]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(data=(y_val),alpha=0.9)
ax = sns.barplot(data=y_val1, alpha=0.7)
ax.set_xticklabels(df10['combined'], rotation='vertical', fontsize=10)
ax.set(xlabel='States',
       ylabel='count of p_values that are significant per state VS California',
       title='Resulting P values'
       )
ax.text(2, 9, "PA = ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "blue",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(2.8, 10,"Other States= ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "orange",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(2.8, 11,"PA Overlap = ♦", 
       fontsize = 10,          # Size
       fontstyle = "oblique",  # Style
       color = "brown",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 

## 11/14/2023 USE THIS
Simpler Is better. So just loc the result dataframe and do the lengths of each. 
The code right below takes the length of the state names list and does.. something. \
I guess sorts and counts and so I have to append those counts to some kind of list. 

But now I can visualize the data properly. By taking everything out of the set of tuples and making a bar graph like the one for the original data

## TO DO 11/15/2023 The below is for unequal var
visualize the p values for texas in front of (or behind) every other state
 

In [ ]:
t_test_unequal_var_2020

In [ ]:
prints=t_test_unequal_var_2020['state2']  # get the state all by itself
prints=prints.unique() 
lists1=[]
for i in prints:   
	abc = df.loc[df['state2']==i]
	abcd = abc[abc['p-value'] < 0.05]
	defg = abc[abc['p-value'] > 0.05]
	lists1.append(['TX',len(abcd),i,len(defg)])
	set_of_tuples = {tuple(inner_list) for inner_list in lists1}


In [ ]:
df10 = pd.DataFrame(set_of_tuples)
df10['combined'] = df10[0]+'/'+df10[2]

In [ ]:
df10.head(2)

In [ ]:
X_1 = range(len(set_of_tuples))
x_val= [x[0] for x in set_of_tuples]
y_val = [x[1] for x in set_of_tuples]
x_val1 = [x[2] for x in set_of_tuples]
y_val1 =[x[3] for x in set_of_tuples]

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(data=(y_val),alpha=0.9)
ax = sns.barplot(data=y_val1, alpha=0.7)
ax.set_xticklabels(df10['combined'], rotation='vertical', fontsize=10)
ax.set(xlabel='States',
       ylabel='count of p_values that are significant per state VS Texas',
       title='Resulting P values'
       )
ax.text(2.2, 20, " Texas = ♦", 
       fontsize = 12,          # Size
       fontstyle = "oblique",  # Style
       color = "blue",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(4, 19,"All other States = ♦", 
       fontsize = 12,          # Size
       fontstyle = "oblique",  # Style
       color = "orange",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 
ax.text(3.8, 18,"Texas Overlap = ♦", 
       fontsize = 12,          # Size
       fontstyle = "oblique",  # Style
       color = "brown",          # Color
       ha = "center", # Horizontal alignment
       va = "center") # Vertical alignment 

# looking at the dataframe above 

In [ ]:
#

# 11/15/2023 
Looking over the data, when compared about half of the values have significane. 

## Code right above is the list organization code. 

In [ ]:
city_counts1 = s2020.groupby('StateAbbr')['LocationName'].count()
city_counts1 = city_counts1.sort_values(ascending=False)
city_counts2 = s2021.groupby('StateAbbr')['LocationName'].count()
city_counts2 = city_counts2.sort_values(ascending=False)
city_counts3 = s2022.groupby('StateAbbr')['LocationName'].count()
city_counts3 = city_counts3.sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
ax = sns.barplot(city_counts1,palette="Greens")

ax.set(xlabel="State",ylabel ="Number of locations ",title="Locations per State 2020")

# The above is all well and good. (11/10/2023)
But it still does not answer my question. Just states that they are different. 
So I am going to attempt a T test on Monday. 


## Working down here for correlations as of 11/15/2023

Trying to organize my values and decide if the values should be dropped. The below code filtered out all of the combinations that had corr values that were not significant. As far as how to move forward. Not sure